## CS140 - Group B


In [0]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.base import TransformerMixin
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.naive_bayes import BernoulliNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report 
from sklearn.ensemble import RandomForestClassifier
from plotly.offline import iplot
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split as tts

In [2]:
import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

In [0]:
TAGS = ['CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNS', 'NNP', 'NNPS', 'PDT', 'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP', 'WP$', 'WRB']
MODALS = ['will', 'would', 'can', 'could', 'may', 'might']
TEMPS = ['before', 'after', 'since']

In [0]:
df = pd.read_csv('https://raw.githubusercontent.com/YonglinWang-Brandeis/cs140-groupB-annotation-files/master/code/features_gold.csv')

In [31]:
df.head()


,POS1,POS2,BEFORE_1,AFTER_1,BEFORE_2,AFTER_2,DISTANCE,MODAL,TEMP_BEFORE,TEMP_AFTER,TEMP_SINCE,SYN,DERIVATION,RELATION
0,VBD,VBG,NNP,PRP,VBD,NNS,3,0,0.0,0.0,0.0,0,0,after
1,VBZ,VBD,EX,DT,NNP,PRP,6,0,0.0,0.0,0.0,0,0,vague
2,VBZ,VBD,EX,DT,NNS,NNP,16,0,0.0,0.0,0.0,0,0,vague
3,VBZ,VBN,EX,DT,RB,RBR,42,0,0.0,0.0,0.0,1,0,after
4,VBZ,VBP,EX,DT,PRP,JJR,47,0,0.0,0.0,0.0,0,0,after


In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 662 entries, 0 to 661
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   POS1         662 non-null    object 
 1   POS2         662 non-null    object 
 2   BEFORE_1     662 non-null    object 
 3   AFTER_1      662 non-null    object 
 4   BEFORE_2     662 non-null    object 
 5   AFTER_2      662 non-null    object 
 6   DISTANCE     662 non-null    int64  
 7   MODAL        662 non-null    int64  
 8   TEMP_BEFORE  662 non-null    float64
 9   TEMP_AFTER   662 non-null    float64
 10  TEMP_SINCE   662 non-null    float64
 11  SYN          662 non-null    int64  
 12  DERIVATION   662 non-null    int64  
 13  RELATION     662 non-null    object 
dtypes: float64(3), int64(4), object(7)
memory usage: 72.5+ KB


In [33]:
df.shape

(662, 14)

In [0]:
def encode_labels(label):
    if label == 'before':
        return 0
    elif label == 'after':
        return 1
    elif label == 'simultaneous':
        return 2
    else:
        return 3

In [0]:
df_copy = df.copy()
df['y'] = df_copy.RELATION.apply(encode_labels)

In [37]:

df.drop(df.index[df['y'] == 3], inplace = True)
# df = df.dropna(axis=0,how='any')
df.head()

,POS1,POS2,BEFORE_1,AFTER_1,BEFORE_2,AFTER_2,DISTANCE,MODAL,TEMP_BEFORE,TEMP_AFTER,TEMP_SINCE,SYN,DERIVATION,RELATION,y
0,VBD,VBG,NNP,PRP,VBD,NNS,3,0,0.0,0.0,0.0,0,0,after,1
3,VBZ,VBN,EX,DT,RB,RBR,42,0,0.0,0.0,0.0,1,0,after,1
4,VBZ,VBP,EX,DT,PRP,JJR,47,0,0.0,0.0,0.0,0,0,after,1
5,VBZ,VBZ,EX,DT,WDT,RBR,61,0,0.0,0.0,0.0,1,1,simultaneous,2
8,VBD,VBN,NNP,PRP,JJ,DT,49,0,0.0,0.0,0.0,1,0,after,1


In [0]:
print(list(df.columns) )

['POS1', 'POS2', 'BEFORE_1', 'AFTER_1', 'BEFORE_2', 'AFTER_2', 'DISTANCE', 'MODAL', 'TEMP_BEFORE', 'TEMP_AFTER', 'TEMP_SINCE', 'SYN', 'DERIVATION', 'RELATION', 'y']


In [0]:
y_data = df['y']
x_data = df[['POS1', 'POS2', 'BEFORE_1', 'AFTER_1', 'BEFORE_2', 'AFTER_2', 'DISTANCE', 'MODAL', 'TEMP_BEFORE', 'TEMP_AFTER', 'TEMP_SINCE', 'SYN', 'DERIVATION']]

In [0]:
X_train, X_test, y_train, y_test = train_test_split(x_data, y_data, train_size=0.8, random_state=0)

In [40]:
from collections import Counter
print('dataset shape {}'.format(Counter(y_data)))
print('dataset shape {}'.format(Counter(y_train)))

dataset shape Counter({0: 202, 1: 108, 2: 42})
dataset shape Counter({0: 163, 1: 84, 2: 34})


# Naíve Bayes

In [0]:
pipeline = Pipeline([    
    ('vec', OneHotEncoder(handle_unknown='ignore')),
    ('clf', BernoulliNB()),
])

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

In [49]:

def test_model(X_test,y_test, model):
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_pred, y_test)
    report = classification_report(y_pred, y_test)
    matrix = pd.DataFrame(confusion_matrix(y_test, y_pred),
                columns=['p_BEFORE', 'p_AFTER','p_SIMU'], index=['BEFORE', 'AFTER','SIMU'])
    return accuracy, report, matrix

accuracy, report, matrix = test_model(X_test, y_test, pipeline)
print("the accuracy of the model is {accuracy:.{digits}f}".format(accuracy=accuracy, digits=2))
print()
print(report)
print()
print(matrix)

the accuracy of the model is 0.68

              precision    recall  f1-score   support

           0       0.74      0.72      0.73        40
           1       0.71      0.63      0.67        27
           2       0.25      0.50      0.33         4

    accuracy                           0.68        71
   macro avg       0.57      0.62      0.58        71
weighted avg       0.70      0.68      0.69        71


        p_BEFORE  p_AFTER  p_SIMU
BEFORE        29        8       2
AFTER          7       17       0
SIMU           4        2       2


# Logistic Regression

In [0]:
pipeline = Pipeline([    
    ('vec', OneHotEncoder(handle_unknown='ignore')) ,

  ('clf', LogisticRegression(
      solver='newton-cg',  
      multi_class='multinomial', 
      )),
])


pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

In [51]:

def test_model(X_test,y_test, model):
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_pred, y_test)
    report = classification_report(y_pred, y_test)
    matrix = pd.DataFrame(confusion_matrix(y_test, y_pred),
                columns=['p_BEFORE', 'p_AFTER','p_SIMU'], index=[ 'BEFORE', 'AFTER','SIMU'])
    return accuracy, report, matrix

accuracy, report, matrix = test_model(X_test, y_test, pipeline)
print("the accuracy of the model is {accuracy:.{digits}f}".format(accuracy=accuracy, digits=2))
print()
print(report)
print()
print(matrix)

the accuracy of the model is 0.69

              precision    recall  f1-score   support

           0       0.79      0.76      0.77        41
           1       0.67      0.67      0.67        24
           2       0.25      0.33      0.29         6

    accuracy                           0.69        71
   macro avg       0.57      0.59      0.58        71
weighted avg       0.71      0.69      0.70        71


        p_BEFORE  p_AFTER  p_SIMU
BEFORE        31        5       3
AFTER          7       16       1
SIMU           3        3       2


# Random Forest

In [0]:
pipeline = Pipeline([    
    ('vec', OneHotEncoder(handle_unknown='ignore')) ,
  ('clf', RandomForestClassifier(n_estimators = 100,
                        min_samples_split = 2,
                        max_depth = 202,))
])


pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

In [65]:

def test_model(X_test,y_test, model):
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_pred, y_test)
    report = classification_report(y_pred, y_test)
    matrix = pd.DataFrame(confusion_matrix(y_test, y_pred),
                columns=[ 'p_BEFORE', 'p_AFTER','p_SIMU'], index=['BEFORE', 'AFTER','SIMU'])
    return accuracy, report, matrix

accuracy, report, matrix = test_model(X_test, y_test, pipeline)
print("the accuracy of the model is {accuracy:.{digits}f}".format(accuracy=accuracy, digits=2))
print()
print(report)
print()
print(matrix)

the accuracy of the model is 0.68

              precision    recall  f1-score   support

           0       0.87      0.67      0.76        51
           1       0.54      0.68      0.60        19
           2       0.12      1.00      0.22         1

    accuracy                           0.68        71
   macro avg       0.51      0.78      0.53        71
weighted avg       0.77      0.68      0.71        71


        p_BEFORE  p_AFTER  p_SIMU
BEFORE        34        5       0
AFTER         11       13       0
SIMU           6        1       1


## GradientBoosting

In [0]:
from sklearn.ensemble import GradientBoostingClassifier

In [57]:
pipeline = Pipeline([    
  ('vec', OneHotEncoder(handle_unknown='ignore')) ,
  ('clf', GradientBoostingClassifier())
])

from collections import Counter
print('dataset shape {}'.format(Counter(y_train)))
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

def test_model(X_test,y_test, model):
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_pred, y_test)
    report = classification_report(y_pred, y_test)
    matrix = pd.DataFrame(confusion_matrix(y_test, y_pred),
                columns=[ 'p_BEFORE', 'p_AFTER','p_SIMU'], index=['BEFORE', 'AFTER','SIMU'])
    return accuracy, report, matrix

accuracy, report, matrix = test_model(X_test, y_test, pipeline)
print("the accuracy of the model is {accuracy:.{digits}f}".format(accuracy=accuracy, digits=2))
print()
print(report)
print()
print(matrix)

dataset shape Counter({0: 163, 1: 84, 2: 34})
the accuracy of the model is 0.70

              precision    recall  f1-score   support

           0       0.82      0.74      0.78        43
           1       0.67      0.67      0.67        24
           2       0.25      0.50      0.33         4

    accuracy                           0.70        71
   macro avg       0.58      0.64      0.59        71
weighted avg       0.74      0.70      0.72        71


        p_BEFORE  p_AFTER  p_SIMU
BEFORE        32        6       1
AFTER          7       16       1
SIMU           4        2       2
